In [1]:
from gurobi import *
import numpy as np
import pandas as pd
from scipy.spatial import distance
from itertools import chain, combinations

# PDSTSP


In [2]:
class Data:
    def __init__(self):
        self.customerNum = 0 
        self.nodeNum     = 0 
        self.droneNum    = 2
        self.cities      = []
        self.cor_X       = [] 
        self.cor_Y       = [] 
        self.serviceTime = [] 
        self.disMatrix   = [[]]
        self.dt          = None
        self.i_pot = None
        self.cus_can_served_by_drone = None
        self.drone_distances = None
        self.truck_distances = None
        
      
        

    def readData(self, path):
        self.dt = pd.read_csv(path, header = None).to_numpy()[:-1]
        self.customerNum = len(self.dt)
        self.i_pot = self.dt[0, 1:3]
        self.nodeNum = self.customerNum + 2 
        
        self.cities = [self.dt[i, 0] for i in range(len(self.dt))]
        
        self.cus_can_served_by_drone = [i for i in range(len(self.dt)) if self.dt[i, 3] == 0]
        
        self.drone_distances = [distance.euclidean((self.dt[i, 1:3]), self.i_pot)
                                if self.dt[i, 3] == 0 else float('inf')
                                for i in range(len(self.dt))]
        self.truck_distances = [[distance.cityblock(self.dt[i, 1:3], self.dt[j, 1:3])
                                 for i in range(len(self.dt))] for j in range(len(self.dt))]
    
    

        
       


In [3]:
data = Data()

data.readData("20140813T111857.csv")
print(data.dt)

print(data.customerNum)

[[ 0.   0.   0.   0.4]
 [ 1.   2.1  2.4  0. ]
 [ 2.   3.8  0.6  1. ]
 [ 3.   0.3  6.6  0. ]
 [ 4.   0.4  3.2  0. ]
 [ 5.   0.6  3.1  0. ]
 [ 6.   2.3 11.9  0. ]
 [ 7.   0.3  9.9  0. ]
 [ 8.   0.8  2.5  0. ]
 [ 9.   3.6 10.7  0. ]]
10


In [4]:
print(data.dt[1, 1:3])

[2.1 2.4]


In [5]:
print(data.cities)

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]


In [6]:
print(data.truck_distances)

[[0.0, 4.5, 4.3999999999999995, 6.8999999999999995, 3.6, 3.7, 14.2, 10.200000000000001, 3.3, 14.299999999999999], [4.5, 0.0, 3.4999999999999996, 5.999999999999999, 2.5000000000000004, 2.2, 9.7, 9.3, 1.4000000000000001, 9.799999999999999], [4.3999999999999995, 3.4999999999999996, 0.0, 9.5, 6.0, 5.699999999999999, 12.8, 12.8, 4.9, 10.299999999999999], [6.8999999999999995, 5.999999999999999, 9.5, 0.0, 3.4999999999999996, 3.7999999999999994, 7.300000000000001, 3.3000000000000007, 4.6, 7.4], [3.6, 2.5000000000000004, 6.0, 3.4999999999999996, 0.0, 0.30000000000000004, 10.6, 6.8, 1.1, 10.7], [3.7, 2.2, 5.699999999999999, 3.7999999999999994, 0.30000000000000004, 0.0, 10.5, 7.1000000000000005, 0.8000000000000002, 10.6], [14.2, 9.7, 12.8, 7.300000000000001, 10.6, 10.5, 0.0, 4.0, 10.9, 2.5000000000000013], [10.200000000000001, 9.3, 12.8, 3.3000000000000007, 6.8, 7.1000000000000005, 4.0, 0.0, 7.9, 4.1], [3.3, 1.4000000000000001, 4.9, 4.6, 1.1, 0.8000000000000002, 10.9, 7.9, 0.0, 11.0], [14.2999999

In [7]:
print(data.drone_distances)

[inf, 3.189043743820395, inf, 6.606814663663572, 3.22490309931942, 3.157530680769389, 12.120231020900551, 9.904544411531507, 2.6248809496813377, 11.289375536317321]


In [8]:
model = Model("PDSTSP")

Academic license - for non-commercial use only - expires 2021-07-04
Using license file /home/quanghuy205/gurobi.lic


In [9]:
#SET
V = [i for i in range(data.customerNum)]
C = [i for i in range (1, data.customerNum)]
U = [k for k in range(data.droneNum)]

C_U = data.cus_can_served_by_drone


C_truck = {(i,j): data.truck_distances[i][j] for i in V for j in V}
C_drone = data.drone_distances

#Decision variables
# z_i = 0: if cus i is visited by vehicle, = 1 if visited by drones
# x_ij if (i->j) in vehicle tour
# # y_im = 1 if cus i assigned to drone m ()

x = [[[] for i in V] for j in V]  
y = [[[] for i in V] for k in U] 



In [10]:
#1
#completion time
alpha = model.addVar(0.0, GRB.INFINITY, 1.0, GRB.CONTINUOUS, "traveltime")
expr = LinExpr(0)
expr.addTerms(1.0, alpha)
model.setObjective(expr, GRB.MINIMIZE)
expr.clear()

In [11]:
#2
for i in V:
    for j in V:
            x[i][j] = model.addVar(0.0, 1.0, 0.0, GRB.BINARY, "x%d,%d" %(i,j))
            expr = LinExpr(0)
            expr.addTerms(data.truck_distances[i][j], x[i][j])
    expr.clear()
model.addConstr(alpha >= expr, "truckTime")
expr.clear()


In [12]:
#3
for k in U:
    expr = LinExpr(0)
    for i in C_U:
        y[k][i] = model.addVar(0.0, 1.0, 0.0, GRB.BINARY, "y%d,%d" %(k,i))
        expr.addTerms(data.drone_distances[i], y[k][i])
        
    model.addConstr(alpha >= expr, "dronetime")
    expr.clear()
    
model.getVars()

[]

In [13]:
#4
for j in C:
    
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    
    for i in V:
        expr1.addTerms(1.0, x[i][j])
    
    for k in U:
        if j in C_U:
            expr2.addTerms(1.0, y[k][j])
    
    model.addConstr(expr1 + expr2 == 1, "served customer once")
    expr1.clear()
    expr2.clear()

 


In [14]:
#5
for i in C:
    
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    
    for j in V:
        expr1.addTerms(1.0, x[i][j])
    
    for k in U:
        if i in C_U:
            expr2.addTerms(1.0, y[k][i])
    
    model.addConstr(expr1 + expr2 == 1, "served customer once")
    expr1.clear()
    expr2.clear()

In [15]:
#6
for i in V:
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    
    for j in V:
        expr1.addTerms(1.0, x[j][i])
    for h in V:
        expr2.addTerms(1.0, x[i][h])
    model.addConstr(expr1 == expr2, "flow conservation")
    expr1.clear()
    expr2.clear()
expr1.clear()
expr2.clear()


In [16]:
#get all subtours
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

S = list(powerset(range(len(data.cities))))
# The first element of the list is the empty set and the last element is the full set, hence we remove them.
S = S[1:(len(S))]

print(len(S))
import sys
print(sys.getsizeof(S)/1024/1024," GB")
# print(S)
S[0]

1023
0.0078582763671875  GB


(0,)

In [17]:
for s in S:
    expr1 = LinExpr(0)
    expr2 = LinExpr(0)
    
    for i in s:
        for j in V:
            if j not in S:
                expr1.addTerms(1.0, x[i][j])
    
    for i in s:
        for k in U:
            if i in C_U:
                expr2.addTerms(1.0, y[k][i])
            
    model.addConstr(expr1 + expr2 >= 1)
    expr1.clear()
    expr2.clear()


In [18]:
model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 1054 rows, 117 columns and 59803 nonzeros
Model fingerprint: 0x9cbfc5bb
Variable types: 1 continuous, 116 integer (116 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 11.2893755
Presolve removed 1026 rows and 18 columns
Presolve time: 0.02s
Presolved: 28 rows, 99 columns, 360 nonzeros
Found heuristic solution: objective 0.0000000
Variable types: 0 continuous, 99 integer (99 binary)

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 12 (of 12 available processors)

Solution count 2: 0 11.2894 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%


In [19]:
model.ObjVal

0.0

In [20]:
model.printAttr('X')


    Variable            X 
-------------------------
        x0,0            1 
        x0,6            1 
        x1,5            1 
        x2,1            1 
        x3,0            1 
        x4,8            1 
        x5,7            1 
        x6,3            1 
        x7,4            1 
        x8,2            1 
        x9,9            1 


In [21]:
model.getVars()

[<gurobi.Var traveltime (value 0.0)>,
 <gurobi.Var x0,0 (value 1.0)>,
 <gurobi.Var x0,1 (value 0.0)>,
 <gurobi.Var x0,2 (value 0.0)>,
 <gurobi.Var x0,3 (value 0.0)>,
 <gurobi.Var x0,4 (value 0.0)>,
 <gurobi.Var x0,5 (value 0.0)>,
 <gurobi.Var x0,6 (value 1.0)>,
 <gurobi.Var x0,7 (value 0.0)>,
 <gurobi.Var x0,8 (value 0.0)>,
 <gurobi.Var x0,9 (value 0.0)>,
 <gurobi.Var x1,0 (value 0.0)>,
 <gurobi.Var x1,1 (value 0.0)>,
 <gurobi.Var x1,2 (value 0.0)>,
 <gurobi.Var x1,3 (value 0.0)>,
 <gurobi.Var x1,4 (value 0.0)>,
 <gurobi.Var x1,5 (value 1.0)>,
 <gurobi.Var x1,6 (value 0.0)>,
 <gurobi.Var x1,7 (value 0.0)>,
 <gurobi.Var x1,8 (value 0.0)>,
 <gurobi.Var x1,9 (value 0.0)>,
 <gurobi.Var x2,0 (value 0.0)>,
 <gurobi.Var x2,1 (value 1.0)>,
 <gurobi.Var x2,2 (value 0.0)>,
 <gurobi.Var x2,3 (value 0.0)>,
 <gurobi.Var x2,4 (value 0.0)>,
 <gurobi.Var x2,5 (value 0.0)>,
 <gurobi.Var x2,6 (value 0.0)>,
 <gurobi.Var x2,7 (value 0.0)>,
 <gurobi.Var x2,8 (value 0.0)>,
 <gurobi.Var x2,9 (value 0.0)>,
 <